# 🎵 Music Source Separation Tool
### Powered by Demucs v4 with GPU Acceleration

---

## Features
- 🚀 **GPU-accelerated** processing for fast separation
- 🎤 **2-stem mode**: Vocals + Instrumental
- 🎸 **4-stem mode**: Vocals, Drums, Bass, Other
- 🌐 **Browser-based UI** with shareable link
- 📥 **Download** individual stems or all as ZIP

## Instructions
1. **Enable GPU**: `Runtime` → `Change runtime type` → `GPU`
2. **Run all cells** in order (Ctrl+Enter or ▶️ button)
3. **Click the Gradio link** that appears at the bottom
4. **Upload audio** and configure settings
5. **Download** your separated stems

## Supported Formats
- MP3, WAV, FLAC, OGG, M4A

## Limitations
- Maximum audio length: **7 minutes**
- Processing time: ~30-90 seconds per song (GPU)

---

# Install Dependencies (Code Cell)

In [10]:
#@title 📦 Install Dependencies { display-mode: "form" }
#@markdown This cell installs all required packages. Run once per session.

import subprocess
import sys

print("=" * 60)
print("🔧 INSTALLING DEPENDENCIES")
print("=" * 60)

# Install core packages
packages = [
    "demucs",
    "gradio>=4.0.0",
    "pydub",
    "ffmpeg-python",
    "torchcodec",      # ✅ ADD THIS - fixes the torchaudio save error
    "soundfile"        # ✅ ADD THIS - backup audio backend
]

for package in packages:
    print(f"\n📦 Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

# Install ffmpeg system package
print("\n🔧 Installing FFmpeg...")
subprocess.run(["apt-get", "-qq", "update"], capture_output=True)
subprocess.run(["apt-get", "-qq", "install", "-y", "ffmpeg"], capture_output=True)

# Install libsndfile for soundfile backend
print("\n🔧 Installing libsndfile...")
subprocess.run(["apt-get", "-qq", "install", "-y", "libsndfile1"], capture_output=True)

print("\n" + "=" * 60)
print("✅ ALL DEPENDENCIES INSTALLED SUCCESSFULLY!")
print("=" * 60)

🔧 INSTALLING DEPENDENCIES

📦 Installing demucs...

📦 Installing gradio>=4.0.0...

📦 Installing pydub...

📦 Installing ffmpeg-python...

📦 Installing torchcodec...

📦 Installing soundfile...

🔧 Installing FFmpeg...

🔧 Installing libsndfile...

✅ ALL DEPENDENCIES INSTALLED SUCCESSFULLY!


# Imports and Configuration (Code Cell)

In [11]:
#@title 🔧 Imports and Device Configuration { display-mode: "form" }
#@markdown Configures GPU/CPU and creates necessary directories.

import os
import sys
import torch
import torchaudio
import subprocess
import shutil
import zipfile
import time
import tempfile
import gc
from pathlib import Path
from typing import Tuple, List, Optional, Dict
import gradio as gr
from pydub import AudioSegment
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# ============================================
# DEVICE CONFIGURATION
# ============================================
print("=" * 60)
print("🖥️  SYSTEM CONFIGURATION")
print("=" * 60)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🔹 PyTorch Version: {torch.__version__}")
print(f"🔹 Device: {DEVICE.upper()}")

if DEVICE == "cuda":
    print(f"🔹 GPU: {torch.cuda.get_device_name(0)}")
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🔹 GPU Memory: {gpu_mem:.2f} GB")
    print("\n✅ GPU acceleration ENABLED - Processing will be fast!")
else:
    print("\n⚠️  GPU not available - Using CPU (slower processing)")
    print("💡 Tip: Go to Runtime → Change runtime type → GPU")

# ============================================
# DIRECTORY SETUP
# ============================================
OUTPUT_DIR = Path("/content/output")
TEMP_DIR = Path("/content/temp")

# Clean and create directories
for dir_path in [OUTPUT_DIR, TEMP_DIR]:
    if dir_path.exists():
        shutil.rmtree(dir_path)
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"\n🔹 Output Directory: {OUTPUT_DIR}")
print(f"🔹 Temp Directory: {TEMP_DIR}")

# ============================================
# CONSTANTS
# ============================================
MAX_DURATION_SECONDS = 7 * 60  # 7 minutes
SAMPLE_RATE = 44100
SUPPORTED_FORMATS = {'.mp3', '.wav', '.flac', '.ogg', '.m4a', '.aac', '.wma'}

print("\n" + "=" * 60)
print("✅ CONFIGURATION COMPLETE")
print("=" * 60)

🖥️  SYSTEM CONFIGURATION

🔹 PyTorch Version: 2.9.0+cu126
🔹 Device: CUDA
🔹 GPU: Tesla T4
🔹 GPU Memory: 15.83 GB

✅ GPU acceleration ENABLED - Processing will be fast!

🔹 Output Directory: /content/output
🔹 Temp Directory: /content/temp

✅ CONFIGURATION COMPLETE


#  Pre-download Models (Code Cell)

In [12]:
#@title 📥 Pre-download Demucs Models { display-mode: "form" }
#@markdown Downloads models in advance to avoid issues during separation.

import torch
print("=" * 60)
print("📥 PRE-DOWNLOADING DEMUCS MODELS")
print("=" * 60)
print("\nThis may take a minute on first run...\n")

# Import demucs and pre-load models
try:
    from demucs.pretrained import get_model
    from demucs.apply import apply_model

    # Download htdemucs model (most commonly used)
    print("📦 Downloading htdemucs model...")
    model_htdemucs = get_model('htdemucs')
    print("   ✅ htdemucs ready!")

    # Move to device to verify it works
    model_htdemucs.to(DEVICE)
    print(f"   ✅ Model loaded on {DEVICE.upper()}")

    # Get model info
    print(f"\n📊 Model Info:")
    print(f"   • Sources: {model_htdemucs.sources}")
    print(f"   • Sample Rate: {model_htdemucs.samplerate}")
    print(f"   • Audio Channels: {model_htdemucs.audio_channels}")

    # Store for later use
    PRELOADED_MODEL = model_htdemucs
    MODEL_LOADED = True

    # Clear memory
    del model_htdemucs
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

except Exception as e:
    print(f"⚠️ Could not pre-load model: {e}")
    print("   Models will be downloaded during first separation.")
    MODEL_LOADED = False
    PRELOADED_MODEL = None

print("\n" + "=" * 60)
print("✅ MODEL PREPARATION COMPLETE")
print("=" * 60)

📥 PRE-DOWNLOADING DEMUCS MODELS

This may take a minute on first run...

📦 Downloading htdemucs model...
   ✅ htdemucs ready!
   ✅ Model loaded on CUDA

📊 Model Info:
   • Sources: ['drums', 'bass', 'other', 'vocals']
   • Sample Rate: 44100
   • Audio Channels: 2

✅ MODEL PREPARATION COMPLETE


# Helper Functions (Code Cell)

In [13]:
#@title 🛠️ Helper Functions { display-mode: "form" }
#@markdown Core utility functions for audio processing.

def clean_directories():
    """Clean output and temp directories"""
    for dir_path in [OUTPUT_DIR, TEMP_DIR]:
        if dir_path.exists():
            shutil.rmtree(dir_path)
        dir_path.mkdir(parents=True, exist_ok=True)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def get_audio_duration(file_path: str) -> float:
    """Get audio duration in seconds using pydub."""
    try:
        audio = AudioSegment.from_file(file_path)
        return len(audio) / 1000.0
    except Exception as e:
        print(f"Error reading audio duration: {e}")
        return -1


def get_audio_info(file_path: str) -> Dict:
    """Get detailed audio information."""
    try:
        audio = AudioSegment.from_file(file_path)
        return {
            "duration": len(audio) / 1000.0,
            "channels": audio.channels,
            "sample_rate": audio.frame_rate,
            "sample_width": audio.sample_width,
            "format": Path(file_path).suffix.lower()
        }
    except Exception as e:
        return {"error": str(e)}


def validate_audio_file(file_path: str) -> Tuple[bool, str]:
    """Validate uploaded audio file."""
    if not file_path:
        return False, "❌ No file uploaded. Please upload an audio file."

    if not os.path.exists(file_path):
        return False, "❌ File not found. Please try uploading again."

    # Check file extension
    ext = Path(file_path).suffix.lower()
    if ext not in SUPPORTED_FORMATS:
        return False, f"❌ Unsupported format: {ext}\nSupported: {', '.join(SUPPORTED_FORMATS)}"

    # Check file size (max 100MB)
    file_size = os.path.getsize(file_path) / (1024 * 1024)
    if file_size > 100:
        return False, f"❌ File too large: {file_size:.1f}MB (max 100MB)"

    # Check duration
    duration = get_audio_duration(file_path)
    if duration < 0:
        return False, "❌ Could not read audio file. It may be corrupted."

    if duration < 1:
        return False, "❌ Audio too short. Minimum length is 1 second."

    if duration > MAX_DURATION_SECONDS:
        return False, f"❌ Audio too long: {duration/60:.1f} minutes\nMaximum allowed: 7 minutes"

    return True, f"✅ Valid audio file ({duration:.1f}s, {file_size:.1f}MB)"


def convert_to_wav(input_path: str, output_path: str) -> Tuple[bool, str]:
    """Convert any audio format to WAV (44.1kHz, stereo)."""
    try:
        audio = AudioSegment.from_file(input_path)

        # Convert to stereo if mono
        if audio.channels == 1:
            audio = audio.set_channels(2)

        # Set sample rate to 44.1kHz
        audio = audio.set_frame_rate(SAMPLE_RATE)

        # Export as WAV
        audio.export(output_path, format="wav")

        return True, f"✅ Converted to WAV: {os.path.basename(output_path)}"
    except Exception as e:
        return False, f"❌ Conversion failed: {str(e)}"


def create_zip_archive(files: List[str], zip_path: str) -> str:
    """Create a ZIP archive from multiple files."""
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in files:
            if file_path and os.path.exists(file_path):
                arcname = os.path.basename(file_path)
                zipf.write(file_path, arcname)
    return zip_path


def format_duration(seconds: float) -> str:
    """Format seconds to MM:SS string"""
    minutes = int(seconds // 60)
    secs = int(seconds % 60)
    return f"{minutes}:{secs:02d}"


print("✅ Helper functions loaded successfully!")

✅ Helper functions loaded successfully!


# Core Separation Engine (Code Cell)

In [14]:
#@title 🎵 Core Separation Engine (Fixed) { display-mode: "form" }

import numpy as np
from scipy.io import wavfile
import soundfile as sf  # ✅ ADD THIS
from demucs.pretrained import get_model
from demucs.apply import apply_model
# ❌ REMOVE: from demucs.audio import save_audio

def load_audio_scipy(path, target_sr=44100):
    """Load audio using scipy (avoids torchcodec issue)"""
    sr, wav = wavfile.read(path)

    # Convert to float32 and normalize
    if wav.dtype == np.int16:
        wav = wav.astype(np.float32) / 32768.0
    elif wav.dtype == np.int32:
        wav = wav.astype(np.float32) / 2147483648.0
    elif wav.dtype != np.float32:
        wav = wav.astype(np.float32)

    # Handle mono/stereo (shape: samples or samples x channels)
    if wav.ndim == 1:
        wav = np.stack([wav, wav], axis=0)  # mono to stereo
    else:
        wav = wav.T  # (samples, channels) -> (channels, samples)

    # Ensure exactly 2 channels
    if wav.shape[0] == 1:
        wav = np.concatenate([wav, wav], axis=0)
    elif wav.shape[0] > 2:
        wav = wav[:2]

    return torch.from_numpy(wav.copy()), sr


def save_audio_sf(wav_tensor, path, samplerate):
    """✅ Save audio using soundfile (avoids torchcodec issue)"""
    # Convert tensor to numpy
    wav_np = wav_tensor.numpy()

    # Transpose: [channels, samples] -> [samples, channels]
    if wav_np.ndim == 2:
        wav_np = wav_np.T

    # Clip to prevent distortion
    wav_np = np.clip(wav_np, -1.0, 1.0)

    # Save using soundfile
    sf.write(path, wav_np, samplerate)


def separate_audio_demucs(input_wav, model_name="htdemucs", two_stems=False, device="cuda"):
    """Separate audio using Demucs Python API"""
    output_files = {}

    try:
        model = get_model(model_name)
        model.to(device)
        model.eval()

        # Load audio using scipy (no torchcodec needed)
        wav, sr = load_audio_scipy(input_wav)

        # Resample if needed
        if sr != model.samplerate:
            wav = torch.nn.functional.interpolate(
                wav.unsqueeze(0),
                scale_factor=model.samplerate/sr,
                mode='linear',
                align_corners=False
            ).squeeze(0)

        wav = wav.unsqueeze(0).to(device)

        with torch.no_grad():
            sources = apply_model(model, wav, device=device, progress=True, num_workers=0)

        sources = sources[0]

        track_name = Path(input_wav).stem
        track_output_dir = OUTPUT_DIR / model_name / track_name
        track_output_dir.mkdir(parents=True, exist_ok=True)

        if two_stems:
            vocals_idx = model.sources.index("vocals")
            vocals_path = track_output_dir / "vocals.wav"
            save_audio_sf(sources[vocals_idx].cpu(), str(vocals_path), model.samplerate)  # ✅ CHANGED
            output_files["vocals"] = str(vocals_path)

            accompaniment = sum(sources[i] for i in range(len(model.sources)) if i != vocals_idx)
            acc_path = track_output_dir / "no_vocals.wav"
            save_audio_sf(accompaniment.cpu(), str(acc_path), model.samplerate)  # ✅ CHANGED
            output_files["no_vocals"] = str(acc_path)
        else:
            for i, name in enumerate(model.sources):
                out_path = track_output_dir / f"{name}.wav"
                save_audio_sf(sources[i].cpu(), str(out_path), model.samplerate)  # ✅ CHANGED
                output_files[name] = str(out_path)

        del model, sources, wav
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return True, output_files, ""

    except Exception as e:
        import traceback
        return False, {}, f"{e}\n{traceback.format_exc()}"


def process_separation(audio_file, mode, stem_count, progress=gr.Progress()):
    """Main Gradio processing function"""
    start_time = time.time()

    try:
        progress(0.1, desc="🔍 Validating...")
        is_valid, msg = validate_audio_file(audio_file)
        if not is_valid:
            return None, None, None, None, None, msg

        progress(0.15, desc="🧹 Preparing...")
        clean_directories()

        progress(0.2, desc="🔄 Converting...")
        temp_wav = TEMP_DIR / "input.wav"
        success, msg = convert_to_wav(audio_file, str(temp_wav))
        if not success:
            return None, None, None, None, None, msg

        progress(0.3, desc=f"🚀 Separating on {DEVICE.upper()}...")
        two_stems = "2 stems" in stem_count

        success, output_files, error = separate_audio_demucs(
            str(temp_wav), "htdemucs", two_stems, DEVICE
        )

        if not success:
            return None, None, None, None, None, f"❌ Failed: {error}"

        progress(0.9, desc="📦 Creating ZIP...")
        all_files = list(output_files.values())
        zip_path = OUTPUT_DIR / "stems.zip"
        create_zip_archive(all_files, str(zip_path))

        elapsed = time.time() - start_time
        status = f"✅ **Done!** {elapsed:.1f}s | {len(all_files)} stems | {DEVICE.upper()}"

        return (
            output_files.get("vocals"),
            output_files.get("no_vocals") or output_files.get("drums"),
            output_files.get("bass"),
            output_files.get("other"),
            str(zip_path),
            status
        )

    except Exception as e:
        return None, None, None, None, None, f"❌ Error: {e}"

    finally:
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

print("✅ Separation engine ready!")

✅ Separation engine ready!


# Gradio User Interface (Code Cell)

In [15]:
#@title 🎨 Build Gradio Interface { display-mode: "form" }
#@markdown Creates the beautiful web interface.

def create_gradio_interface():
    """Create and configure the Gradio interface"""

    # Custom CSS for styling
    custom_css = """
    .gradio-container {
        max-width: 1000px !important;
        margin: auto !important;
    }
    .gr-button-primary {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        border: none !important;
        font-size: 18px !important;
        padding: 12px 24px !important;
    }
    .gr-button-primary:hover {
        transform: translateY(-2px) !important;
        box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4) !important;
    }
    .footer-text {
        text-align: center;
        padding: 20px;
        color: #666;
        font-size: 13px;
    }
    """

    # Build interface using Blocks
    with gr.Blocks(
        title="🎵 Music Source Separation",
        theme=gr.themes.Soft(
            primary_hue="purple",
            secondary_hue="blue",
        ),
        css=custom_css
    ) as interface:

        # ==========================================
        # HEADER SECTION
        # ==========================================
        gr.Markdown("""
        <div style="text-align: center; padding: 20px;">
            <h1>🎵 AI Music Source Separation</h1>
            <h3>Powered by Demucs v4 • GPU Accelerated</h3>
            <p style="color: #666;">
                Separate any song into individual stems: Vocals, Drums, Bass, and Instruments
            </p>
        </div>
        """)

        # Device status banner
        if DEVICE == "cuda":
            gr.Markdown("""
            <div style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
                        color: white; padding: 10px 20px; border-radius: 10px;
                        text-align: center; margin: 10px 0;">
                🎮 <strong>GPU Enabled</strong> - Fast processing available!
            </div>
            """)
        else:
            gr.Markdown("""
            <div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
                        color: white; padding: 10px 20px; border-radius: 10px;
                        text-align: center; margin: 10px 0;">
                ⚠️ <strong>CPU Mode</strong> - Processing will be slower.
                Enable GPU: Runtime → Change runtime type → GPU
            </div>
            """)

        gr.Markdown("---")

        # ==========================================
        # MAIN CONTENT
        # ==========================================
        with gr.Row():
            # LEFT COLUMN - Input Controls
            with gr.Column(scale=1):
                gr.Markdown("### 📤 Upload & Configure")

                audio_input = gr.Audio(
                    label="🎵 Upload Audio File",
                    type="filepath",
                    sources=["upload"],
                )

                gr.Markdown("*Supported: MP3, WAV, FLAC, OGG, M4A (max 7 minutes)*")

                mode_selector = gr.Dropdown(
                    label="🎚️ Quality Mode",
                    choices=[
                        "High Quality (Demucs v4)",
                        "Standard (Demucs v4)"
                    ],
                    value="High Quality (Demucs v4)",
                    info="Both modes use htdemucs model"
                )

                stem_selector = gr.Dropdown(
                    label="🎼 Stem Count",
                    choices=[
                        "2 stems (vocals + instrumental)",
                        "4 stems (vocals, drums, bass, other)"
                    ],
                    value="2 stems (vocals + instrumental)",
                    info="2 stems is faster, 4 stems gives more control"
                )

                separate_button = gr.Button(
                    "🚀 Start Separation",
                    variant="primary",
                    size="lg"
                )

                # Status display
                status_display = gr.Markdown(
                    value="*Ready to process. Upload an audio file to begin.*",
                    label="Status"
                )

        gr.Markdown("---")

        # ==========================================
        # OUTPUT SECTION
        # ==========================================
        gr.Markdown("### 🎧 Separated Stems")
        gr.Markdown("*Preview and download your separated tracks below*")

        with gr.Row():
            with gr.Column(scale=1):
                vocals_output = gr.Audio(
                    label="🎤 Vocals",
                    type="filepath",
                    interactive=False
                )

            with gr.Column(scale=1):
                instrumental_output = gr.Audio(
                    label="🎸 Instrumental / Drums",
                    type="filepath",
                    interactive=False
                )

        with gr.Row():
            with gr.Column(scale=1):
                bass_output = gr.Audio(
                    label="🎸 Bass",
                    type="filepath",
                    interactive=False
                )

            with gr.Column(scale=1):
                other_output = gr.Audio(
                    label="🎹 Other / Instruments",
                    type="filepath",
                    interactive=False
                )

        gr.Markdown("---")

        # Download all section
        gr.Markdown("### 📦 Download All Stems")

        download_zip = gr.File(
            label="Download ZIP Archive",
            file_count="single",
            type="filepath",
            interactive=False
        )

        gr.Markdown("---")

        # ==========================================
        # FOOTER / DISCLAIMER
        # ==========================================
        gr.Markdown(f"""
        <div class="footer-text">
            <h4>⚠️ Legal Disclaimer</h4>
            <p>
                This tool is provided for <strong>educational and personal use only</strong>.<br>
                Users must have the legal rights to any audio they upload and process.<br>
                Do not use this tool for copyright infringement or unauthorized commercial purposes.
            </p>
            <hr style="border: 1px solid #eee; margin: 15px 0;">
            <p>
                Built with ❤️ using <strong>Demucs v4</strong> by Meta Research & <strong>Gradio</strong><br>
                🖥️ Running on: <strong>{'GPU 🎮' if DEVICE == 'cuda' else 'CPU 💻'}</strong>
            </p>
        </div>
        """)

        # ==========================================
        # EVENT HANDLERS
        # ==========================================

        # Main separation button click
        separate_button.click(
            fn=process_separation,
            inputs=[
                audio_input,
                mode_selector,
                stem_selector
            ],
            outputs=[
                vocals_output,
                instrumental_output,
                bass_output,
                other_output,
                download_zip,
                status_display
            ],
            show_progress="full"
        )

        # Validate on upload
        def on_audio_upload(audio_file):
            if audio_file:
                is_valid, msg = validate_audio_file(audio_file)
                return msg
            return "*Ready to process. Upload an audio file to begin.*"

        audio_input.change(
            fn=on_audio_upload,
            inputs=[audio_input],
            outputs=[status_display]
        )

    return interface


# Create the interface
print("🎨 Building Gradio interface...")
demo = create_gradio_interface()
print("✅ Interface created successfully!")

🎨 Building Gradio interface...
✅ Interface created successfully!


# Launch Application (Code Cell)

In [16]:
#@title 🚀 Launch Application { display-mode: "form" }
#@markdown This will start the Gradio server and generate a public link.

print("=" * 60)
print("🚀 LAUNCHING MUSIC SOURCE SEPARATION TOOL")
print("=" * 60)
print()
print("📌 Please wait while the server starts...")
print("📌 A public URL will be generated for sharing")
print()
print("=" * 60)

# Enable queue for progress tracking
demo.queue(max_size=5)

# Launch with public sharing enabled
demo.launch(
    share=True,
    debug=False,
    show_error=True,
    quiet=False
)

🚀 LAUNCHING MUSIC SOURCE SEPARATION TOOL

📌 Please wait while the server starts...
📌 A public URL will be generated for sharing

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ab060a1d3bb25714c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Cleanup

In [17]:
#@title 🧹 Cleanup (Run when done) { display-mode: "form" }
#@markdown This cell cleans up temporary files and frees GPU memory.

import shutil
import gc
import torch

print("🧹 Cleaning up...")

# Clean directories
for dir_path in [OUTPUT_DIR, TEMP_DIR]:
    if dir_path.exists():
        shutil.rmtree(dir_path)
        print(f"   ✓ Removed {dir_path}")

# Clear GPU memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("   ✓ GPU memory cleared")

print("\n✅ Cleanup complete!")

🧹 Cleaning up...
   ✓ Removed /content/output
   ✓ Removed /content/temp
   ✓ GPU memory cleared

✅ Cleanup complete!
